# Streaming

Streaming is a capability that allows AI providers to deliver responses as tokens are generated in real-time,
rather than waiting for the complete response to be finished.
You can visualize it as filling a bucket with water — streaming lets you watch the water level rising in real-time,
instead of placing the bucket somewhere, walking away, and only returning once it's completely full.

Streaming significantly improves the user experience,
as people can immediately see the response forming rather than staring at a loading indicator.
This is especially valuable since synchronous requests can take quite a long time to generate.

Let's implement streaming responses with Kotlin and Spring AI.
For this, we'll need to add dependencies.
In addition to `spring-ai-openai`, we'll need the Kotlin coroutines library to work with `Flow`.

In [1]:
%useLatestDescriptors
%use coroutines
%use spring-ai-openai
USE { dependencies { implementation("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.1") } }

As in the previous notebooks, we'll need an API key.

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

Now let's create a `ChatModel` and pass in the `ChatOptions`:

In [3]:
val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()

val chatCompletion = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Now let's ask the model to generate a short fairy tale about rabbits:

In [4]:
import kotlinx.coroutines.reactive.asFlow

val response: Flow<String> = chatCompletion.stream("Generate me 10 sentences of a fairy tale about rabbits").asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time in a lush, green meadow, there lived a brave little rabbit named Benny who dreamed of exploring the world beyond the tall, golden grass. One sunny morning, Benny discovered a sparkling, hidden path lined with colorful wildflowers that led deep into the Enchanted Forest. As he hopped along the path, he met a wise old tortoise named Tilly, who warned him of a mischievous fox lurking nearby, always on the lookout for a tasty rabbit snack. Undeterred, Benny continued on his adventure, determined to find the legendary Rainbow Carrot said to grant any wish to the one who found it. 

Along the way, Benny befriended a group of playful squirrels who shared their acorn stash and joined him on his quest, forming a bond of friendship that made them stronger together. They danced and sang as they ventured deeper into the forest, unaware of the sly fox watching them from behind a tree. Suddenly, the fox sprang out, but quick-thinking Benny devised a clever plan, leading the fox on a

As you can see, the response starts printing immediately.
If this weren't a streaming response,
we would be waiting approximately 8 seconds (judging by the cell execution time) with nothing happening.

Now let's do the same thing using the `ChatClient`:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val response: Flow<String> = chatClient
    .prompt("Generate me 10 sentences of a fairy tale about rabbits")
    .stream()
    .content()
    .asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time in a lush, green meadow, there lived a clever rabbit named Ruby, who had the softest fur and the brightest eyes. Every morning, she would hop through the dew-kissed grass, gathering golden dandelions to share with her friends. One day, while exploring a hidden glade, Ruby stumbled upon a shimmering pond that sparkled like diamonds under the sun. 

Curious and enchanted, she approached the water and discovered a wise old turtle named Timothy, who told her tales of a magical forest where dreams came true. Eager to find this wondrous place, Ruby gathered her friends—a brave rabbit named Benny and a gentle rabbit called Lily—and together they set off on an adventure. 

As they traveled, they encountered a mischievous fox who tried to trick them, but Ruby outsmarted him with her quick thinking and clever riddles. After a long journey filled with laughter and challenges, they finally reached the enchanted forest, where the trees whispered secrets and flowers sang sweet melod